In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


# Model Evaluation
from sklearn.metrics import (roc_curve, roc_auc_score, f1_score, precision_score, recall_score)

# Cross-validation and model tuning
from sklearn.model_selection import cross_val_score, GridSearchCV

import warnings
warnings.filterwarnings('ignore')


### Reading in CRE Data + Summary Statistics

In [ ]:
loans = pd.read_csv("data/CRE-Loan-Data.csv")
loans.head(3)

In [ ]:
# Dataframe info by column
loans.info()

In [ ]:
# Summary stats for initial numerical attributes
loans.describe()

### Takeaways
- 49.2 months - the average time remaining on the dataset's loans at the time recorded
    - longest-term existing loan is 180 months away
- 99 months - the average loan term
- 9.8% - the approximate percentage of loans in the dataset that defaulted within 12 months
- 2002 - the average year of construction for properties in the dataset
- 1.8 - the average interest coverage ratio (ICR)

#### Recasting datatypes to maximize numerical columns

In [5]:
# Convert 'Rating snapshot date' to datetime
loans['Rating snapshot date'] = pd.to_datetime(loans['Rating snapshot date'])

numericCols = ['Internal rating', 'Loan Balance', 'Interest rate', 'Property value',
                'Net operating income', 'Interest coverage ratio', 'Original term',
                'Months to maturity', 'Tenant turnover', 'Year of construction', 'Default flag']

# Strip values of $, %, or commas and convert to numerics, also convert "(x)" to "-x"
def cleanNumericCols(column):
    return pd.to_numeric(column.replace({'\$':'', ',':'', '%':'', '\(':'-', '\)':''}, regex=True))

for col in numericCols:
    loans[col] = cleanNumericCols(loans[col])

# Convert % rate to proportion
loans.loc[loans['Interest rate'] > 1, 'Interest rate'] /= 100

### Preparing new feature engineered variables `Loan-to-Value (LTV) Ratio`, `Debt Yield`

In [6]:
loans['LoanToValueRatio'] = loans['Loan Balance'] / loans['Property value']
loans['DebtYield'] = loans['Net operating income'] / loans['Loan Balance']

# Interaction Effects
# loans['InterestCoverage-LoanBalance'] = loans['Interest coverage ratio'] * loans['Loan Balance']
# loans['InterestRate-PropertyValue'] = loans['Interest rate'] * loans['Property value']

# Drop rows with infinite values (in case of division by zero)
loans.replace([np.inf, -np.inf], np.nan, inplace=True)
loans.dropna(inplace=True)

In [ ]:
# Default rate by internal rating
loans.pivot_table(index=['Internal rating'], values='Default flag', aggfunc='mean')

#### Calculate & Visualize Default Rate by Internal Rating

In [8]:
grouped = loans.groupby('Internal rating')['Default flag'].agg(['count', 'sum', 'mean']).reset_index()
grouped = grouped.rename(columns={'count': 'Number of Loans', 
                                  'sum': 'Number of Defaults', 'mean': 'Default Rate'})

# grouped.loc[grouped['Default Rate'] < 1, 'Default Rate'] *= 100
grouped['Default Rate'] = grouped['Default Rate'].round(3)
# grouped.to_excel('riskRating-default.xlsx')

In [ ]:
# Plot default rate by internal rating
fig = px.bar(grouped, x='Internal rating', y='Default Rate',
             title='Default Rate by Internal Rating', text='Default Rate')
fig.show()

With the exception of an observed decrease in default rate between loans rated at a 4 and 5, higher internal credit risk ratings seem to correspond to higher default rates, as is expected.

In [ ]:
# ROC curve and AUC for internal rating
fpr, tpr, thresholds = roc_curve(loans['Default flag'], loans['Internal rating'])
auc = roc_auc_score(loans['Default flag'], loans['Internal rating'])

# GenAI created ROC curve
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC curve (AUC = {auc:.2f})'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Baseline (Random Guess)', line=dict(dash='dash')))
fig.update_layout(title='ROC Curve for Internal Rating',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate',
                  showlegend=True)
fig.show()

### Calculate correlation with `Internal rating`

In [ ]:
loansNumeric = loans.select_dtypes(include=[np.number])

corrMatrix = loansNumeric.corr()

corrWithRating = corrMatrix['Internal rating'].sort_values(ascending=False)
corrWithRating

In [ ]:
# Plot correlation with internal rating
corrDF = corrWithRating.drop('Internal rating').reset_index()
corrDF.columns = ['Feature', 'Correlation']
corrDF['Correlation'] = np.round(corrDF['Correlation'], 3)

fig = px.bar(corrDF, x='Feature', y='Correlation',
             title='Correlation with Internal Rating', 
             )
fig.update_layout(xaxis_tickangle=-45)
fig.show()

#### Random Forest regression on `Internal Rating` with dataset's features

In [13]:
Xrating = loans.drop(['Internal rating', 'Default flag', 
                      'Facility ID', 'Rating snapshot date', 'Portfolio'], axis=1)
yrating = loans['Internal rating']

categoricalFeatures = ['Property type']
numericalFeatures = [col for col in Xrating.columns if col not in categoricalFeatures]

# Preprocessing pipelines
numericalTransformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categoricalTransformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessorRating = ColumnTransformer(
    transformers=[
        ('num', numericalTransformer, numericalFeatures),
        ('cat', categoricalTransformer, categoricalFeatures)
    ])


In [ ]:
regressor = Pipeline(steps=[('preprocessor', preprocessorRating),
                            ('regressor', RandomForestRegressor(random_state=123))])

regressor.fit(Xrating, yrating)

### Determine & Plot Feature Importances for Regression on `Internal Rating`

In [ ]:
featureNamesCat = regressor.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categoricalFeatures)
featureNames = np.concatenate([numericalFeatures, featureNamesCat])

importances = regressor.named_steps['regressor'].feature_importances_
featureImportances = pd.Series(importances, index=featureNames).sort_values(ascending=False)
np.round(featureImportances * 100, 3).head(10)

In [ ]:
# Plot feature importances
importanceDf = featureImportances.reset_index()
importanceDf.columns = ['Feature', 'Importance']
importanceDf['Importance'] = np.round(importanceDf['Importance'], 3)

fig = px.bar(importanceDf.head(10), x='Feature', y='Importance',
             title='Top 10 Feature Importances for Internal Rating', 
             )
fig.update_layout(xaxis_tickangle=-45)
fig.show()

Top 3 factors influencing the internal rating:
1. **Interest Coverage Ratio**: 66.8% importance
2. **Property Value**: 8.3% importance
3. **Loan Balance**: 6.7% importance

#### Predictive model on `Default Flag`

In [17]:
# Features and target variable
X = loans.drop(['Default flag', 'Facility ID', 'Rating snapshot date', 'Portfolio'], axis=1)
y = loans['Default flag']

# Categorical and numerical features
categoricalFeatures = ['Property type']
numericalFeatures = [col for col in X.columns if col not in categoricalFeatures + ['Internal rating']]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=123)

In [18]:
# Preprocessing pipelines
numericalTransformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categoricalTransformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numericalTransformer, numericalFeatures + ['Internal rating']),
        ('cat', categoricalTransformer, categoricalFeatures)
    ])

#### Logistic Regression Classifier Model

In [ ]:
# Logistic Regression model
pipelineLR = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', LogisticRegression(max_iter=1000))])

pipelineLR.fit(Xtrain, ytrain)

# Predict probabilities and compute AUC for Logistic Regression
yPredProbaLR = pipelineLR.predict_proba(Xtest)[:,1]
aucLR = roc_auc_score(ytest, yPredProbaLR)
print(f'Logistic Regression AUC: {aucLR:.4f}')

#### Scikit-learn Gradient Boosting Classifier

In [ ]:
pipelineGB = Pipeline(steps=[('preprocessor', preprocessor),
                             ('classifier', GradientBoostingClassifier(random_state=42))])
pipelineGB.fit(Xtrain, ytrain)
yPredProbaGB = pipelineGB.predict_proba(Xtest)[:,1]

scoresGB = cross_val_score(pipelineGB, Xtrain, ytrain, cv=5, scoring='roc_auc')
print(f'Gradient Boosting CV AUC: {np.mean(scoresGB):.4f}')

#### XGBoost Classifier model

In [ ]:
pipelineXGB = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', XGBClassifier(eval_metric='logloss', random_state=42))])
pipelineXGB.fit(Xtrain, ytrain)
yPredProbaXGB = pipelineXGB.predict_proba(Xtest)[:,1]

scoresXGB = cross_val_score(pipelineXGB, Xtrain, ytrain, cv=5, scoring='roc_auc')
print(f'XGBoost CV AUC: {np.mean(scoresXGB):.4f}')

#### Baseline (`Internal Rating`) ROC-AUC Score

In [ ]:
# Compute AUC for Internal Rating on test set
ratingTest = Xtest['Internal rating']
aucInternal = roc_auc_score(ytest, ratingTest)
print(f'Internal Rating AUC: {aucInternal:.4f}')

### Model Comparison Summary

In [ ]:
# Compile the results
modelResults = pd.DataFrame({
    'Model': ['Logistic Regression', 'Gradient Boosting', 'XGBoost'],
    'CV AUC': [np.mean(aucLR), np.mean(scoresGB), np.mean(scoresXGB)]
}).set_index('Model')

print(modelResults.sort_values(by='CV AUC', ascending=False))

#### Hyperparameter tuning on highest performing (by ROC-AUC) models: `Logistic Regression`, `Scikit-Learn Gradient Boosting`, & `XGBoost`

In [ ]:
paramGridLR = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__penalty': ['l2'],
    'classifier__solver': ['lbfgs', 'liblinear']
}

gridSearchLR = GridSearchCV(pipelineLR, paramGridLR, cv=5, scoring='roc_auc')
gridSearchLR.fit(Xtrain, ytrain)

print(f'Best Logistic Regression AUC: {gridSearchLR.best_score_:.4f}')
print(f'Best Parameters: {gridSearchLR.best_params_}')


In [ ]:
paramGridGB = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': [3, 5]
}

gridSearchGB = GridSearchCV(pipelineGB, paramGridGB, cv=5, scoring='roc_auc', n_jobs=-1)
gridSearchGB.fit(Xtrain, ytrain)

print(f'Best Gradient Boosting AUC: {gridSearchGB.best_score_:.4f}')
print(f'Best Parameters: {gridSearchGB.best_params_}')


In [ ]:
paramGridXGB = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': [3, 5],
    'classifier__subsample': [0.8, 1]
}

gridSearchXGB = GridSearchCV(pipelineXGB, paramGridXGB, cv=5, scoring='roc_auc', n_jobs=-1)
gridSearchXGB.fit(Xtrain, ytrain)

print(f'Best XGBoost AUC: {gridSearchXGB.best_score_:.4f}')
print(f'Best Parameters: {gridSearchXGB.best_params_}')


------------------

### Hyperparameter optimized models for `LR`, `GB`, and `XGB` models

In [ ]:
bestPipelineLR = gridSearchLR.best_estimator_
yPredProbaLR = bestPipelineLR.predict_proba(Xtest)[:, 1]
aucLR = roc_auc_score(ytest, yPredProbaLR)
print(f'Logistic Regression Test AUC: {aucLR:.4f}')

In [ ]:
bestPipelineGB = gridSearchGB.best_estimator_
yPredProbaGB = bestPipelineGB.predict_proba(Xtest)[:, 1]
aucGB = roc_auc_score(ytest, yPredProbaGB)
print(f'Gradient Boosting Test AUC: {aucGB:.4f}')

In [ ]:
bestPipelineXGB = gridSearchXGB.best_estimator_
yPredProbaXGB = bestPipelineXGB.predict_proba(Xtest)[:, 1]
aucXGB = roc_auc_score(ytest, yPredProbaXGB)
print(f'XGBoost Test AUC: {aucXGB:.4f}')

In [ ]:
fprLR, tprLR, _ = roc_curve(ytest, yPredProbaLR)
fprGB, tprGB, _ = roc_curve(ytest, yPredProbaGB)
fprXGB, tprXGB, _ = roc_curve(ytest, yPredProbaXGB)
fprInternal, tprInternal, _ = roc_curve(ytest, Xtest['Internal rating'])

# Plot ROC curves
fig = go.Figure()
fig.add_trace(go.Scatter(x=fprInternal, y=tprInternal, mode='lines', name='Internal Rating'))
fig.add_trace(go.Scatter(x=fprLR, y=tprLR, mode='lines', name='Logistic Regression'))
fig.add_trace(go.Scatter(x=fprGB, y=tprGB, mode='lines', name='Scikit-Learn Gradient Boosting'))
fig.add_trace(go.Scatter(x=fprXGB, y=tprXGB, mode='lines', name='XGBoost'))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Baseline (Random Guess)', line=dict(dash='dash')))
fig.update_layout(title='ROC Curves Comparison',
                  xaxis_title='False Positive Rate',
                  yaxis_title='True Positive Rate',
                  showlegend=True)
fig.show()


In [ ]:
featureNamesNum = numericalFeatures + ['Internal rating']
featureNamesCat = (bestPipelineXGB.named_steps['preprocessor'].named_transformers_['cat']
                   .named_steps['onehot'].get_feature_names_out(categoricalFeatures))
featureNames = np.concatenate([featureNamesNum, featureNamesCat])

importancesXGB = bestPipelineXGB.named_steps['classifier'].feature_importances_
featureImportancesXGB = pd.Series(importancesXGB, index=featureNames).sort_values(ascending=False)

featureImportancesXGB.head(10)

In [ ]:
# Plot feature importances from final XGBoost model
importanceXGBdf = featureImportancesXGB.reset_index()
importanceXGBdf.columns = ['Feature', 'Importance']
importanceXGBdf['Importance'] = np.round(importanceXGBdf['Importance'], 3)

fig = px.bar(importanceXGBdf.head(10), x='Feature', y='Importance',
             title='Top 10 Feature Importances from Optimized XGB Classifier Model', text="Importance")
fig.update_layout(xaxis_tickangle=-45)
fig.show()
importanceXGBdf

### Conclusions
- `XGBoost's Gradient Boosting Classifier` with tuned parameters achieves the highest AUC on the test set.
- `Internal rating` is the highest importance predictor for the optimized XGB classifier